# Phi-2 GRPO Fine-tuning on Google Colab

This notebook fine-tunes the Microsoft Phi-2 model using GRPO (Group Relative Policy Optimization) on the OpenAssistant/oasst1 dataset with QLoRA 4-bit quantization.

**Optimized for T4 GPU (16GB VRAM)**

## Setup Instructions

1. Make sure you're using a GPU runtime (Runtime → Change runtime type → GPU → T4)
2. Run all cells in order
3. Optionally mount Google Drive to save checkpoints

In [ ]:
# T4 GPU Configuration: Disable bf16 BEFORE any imports
import os
os.environ["DISABLE_BF16"] = "1"

# Check GPU availability and T4 compatibility
import torch

# Disable TF32 and bf16 at the PyTorch level
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.allow_tf32 = False

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    compute_capability = torch.cuda.get_device_capability(0)
    
    print(f"✅ GPU available: {gpu_name}")
    print(f"GPU Memory: {gpu_memory:.2f} GB")
    print(f"Compute Capability: {compute_capability[0]}.{compute_capability[1]}")
    print(f"CUDA Version: {torch.version.cuda}")
    
    # Check if T4 GPU
    if "T4" in gpu_name:
        print("\n⚠️  T4 GPU detected - Important notes:")
        print("   - bf16 is NOT supported (will use fp16)")
        print("   - 4-bit quantization IS supported")
        print("   - Memory: 16GB (suitable for Phi-2 with QLoRA)")
        print("\n✅ T4 optimizations applied (bf16 disabled, fp16 enabled)")
    
    # Check compute capability for bitsandbytes
    if compute_capability[0] >= 7:
        print("\n✅ GPU supports 4-bit quantization (compute capability >= 7.0)")
    else:
        print("\n⚠️  GPU may have limited support for 4-bit quantization")
else:
    print("❌ No GPU detected. Please enable GPU runtime.")
    print("Go to: Runtime → Change runtime type → GPU → T4")

## 1. Install Dependencies

In [ ]:
# Install all packages in one command to let pip resolve dependencies correctly
!pip install -q transformers accelerate datasets bitsandbytes peft trl pyyaml tqdm

# Verify installations
import torch
import transformers
import accelerate
import bitsandbytes
import peft
import trl

print(f"PyTorch: {torch.__version__}")
print(f"Transformers: {transformers.__version__}")
print(f"Accelerate: {accelerate.__version__}")
print(f"Bitsandbytes: {bitsandbytes.__version__}")
print(f"PEFT: {peft.__version__}")
print(f"TRL: {trl.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")

print("\n✅ Dependencies installed!")

## 2. Mount Google Drive (Optional)

Mount Google Drive to save checkpoints and model files.

In [ ]:
from google.colab import drive

# Uncomment to mount Google Drive
# drive.mount('/content/drive')

# Set output directory (change to your Drive path if mounted)
OUTPUT_DIR = "./outputs"
# OUTPUT_DIR = "/content/drive/MyDrive/phi2-grpo-outputs"  # Uncomment if using Drive

print(f"Output directory: {OUTPUT_DIR}")

## 3. Configuration

Set your training hyperparameters here. These are optimized for T4 GPU.

In [ ]:
# Model and Dataset
MODEL_NAME = "microsoft/phi-2"
DATASET_NAME = "OpenAssistant/oasst1"
LANGUAGE = "en"  # Filter for English conversations

# Training hyperparameters (optimized for T4 GPU 16GB VRAM - targeting 12-13GB usage)
# GRPO generates multiple completions per prompt internally (typically 4-8)
BATCH_SIZE = 2  # Increased from 1 to better utilize GPU
EVAL_BATCH_SIZE = 8  # Must be divisible by GRPO's default generations
GRADIENT_ACCUMULATION_STEPS = 16  # Increased for larger effective batch size (2 * 16 = 32)
LEARNING_RATE = 1e-4
NUM_EPOCHS = 1  # Keep at 1 for initial training
MAX_SEQ_LENGTH = 512  # Increased from 256 for better context
MAX_SAMPLES = 5000  # Increased from 1000 for more training data
# Note: These settings should use approximately 12-13GB of GPU memory

# LoRA parameters (balanced for quality and memory)
LORA_R = 16  # Increased from 8 for better model capacity
LORA_ALPHA = 32  # Increased from 16
LORA_DROPOUT = 0.05

# Output
OUTPUT_DIR = "./outputs"
SAVE_STEPS = 500
LOGGING_STEPS = 10

print("✅ Configuration set (optimized for ~12-13GB GPU usage)!")

## 4. Authenticate with Hugging Face (Optional)

If you want to push the model to Hugging Face Hub, authenticate here.

In [ ]:
from huggingface_hub import login

# Uncomment and add your HF token
login(token="hf_token")

print("✅ Hugging Face authentication (if enabled)")

## 5. Load and Prepare Dataset

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load tokenizer
print(f"Loading tokenizer: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print("✅ Tokenizer loaded")

In [ ]:
# Load dataset
print(f"Loading dataset: {DATASET_NAME}")
dataset = load_dataset(DATASET_NAME, split="train")

if MAX_SAMPLES:
    dataset = dataset.select(range(min(MAX_SAMPLES, len(dataset))))
    print(f"Limited to {len(dataset)} samples")

print(f"✅ Dataset loaded: {len(dataset)} samples")

In [ ]:
# Extract conversation pairs from oasst1 tree structure
def extract_conversations(dataset, language="en"):
    """Extract prompt-completion pairs from oasst1."""
    conversations = []
    
    # Filter for language and approved messages
    filtered = [
        msg for msg in dataset
        if msg.get("lang") == language
        and not msg.get("deleted", False)
        and msg.get("review_result", False)
    ]
    
    message_dict = {msg["message_id"]: msg for msg in filtered}
    root_messages = [msg for msg in filtered if msg.get("parent_id") is None]
    
    def get_thread(message_id, thread=None):
        if thread is None:
            thread = []
        if message_id not in message_dict:
            return thread
        
        msg = message_dict[message_id]
        thread.append(msg)
        
        children = [m for m in filtered if m.get("parent_id") == message_id]
        # Handle None values in rank - use 0 if rank is None or missing
        children.sort(key=lambda x: (x.get("rank") if x.get("rank") is not None else 0, x.get("created_date", "")))
        
        if children:
            return get_thread(children[0]["message_id"], thread)
        return thread
    
    for root in root_messages:
        thread = get_thread(root["message_id"])
        for i in range(len(thread) - 1):
            if thread[i]["role"] == "prompter" and thread[i + 1]["role"] == "assistant":
                conversations.append({
                    "prompt": thread[i]["text"],
                    "completion": thread[i + 1]["text"],
                })
    
    return conversations

print("Extracting conversation pairs...")
conversations = extract_conversations(dataset, LANGUAGE)
print(f"✅ Extracted {len(conversations)} conversation pairs")

In [ ]:
# Format conversations for training
def format_for_training(conversations, tokenizer, max_length=512):
    """Format conversations for GRPO training."""
    formatted = []
    
    for conv in conversations:
        prompt = conv["prompt"]
        completion = conv["completion"]
        
        # Format prompt text (what GRPOTrainer expects)
        prompt_text = f"Human: {prompt}\n\nAssistant: "
        
        # Full text for reference
        text = f"Human: {prompt}\n\nAssistant: {completion}"
        
        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=max_length,
            padding=False,
        )
        
        prompt_tokenized = tokenizer(
            prompt_text,
            truncation=True,
            max_length=max_length,
        )
        
        # GRPOTrainer requires "prompt" field
        formatted.append({
            "prompt": prompt_text,  # Required by GRPOTrainer
            "text": text,  # Full text for reference
            "input_ids": tokenized["input_ids"],
            "attention_mask": tokenized["attention_mask"],
            "prompt_length": len(prompt_tokenized["input_ids"]),
        })
    
    return formatted

print("Formatting conversations...")
from datasets import Dataset
formatted_data = format_for_training(conversations, tokenizer, MAX_SEQ_LENGTH)
train_dataset = Dataset.from_list(formatted_data)

# Split into train/validation
train_dataset = train_dataset.train_test_split(test_size=0.1, seed=42)
print(f"✅ Training samples: {len(train_dataset['train'])}")
print(f"✅ Validation samples: {len(train_dataset['test'])}")

## 6. Load Model with 4-bit Quantization and QLoRA

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

# T4 GPU does NOT support bf16 - must use fp16
# Setup 4-bit quantization with float16 (NOT bfloat16)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # T4 requires float16, NOT bfloat16
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

print(f"Loading model: {MODEL_NAME} with 4-bit quantization (fp16 for T4 GPU)...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16,  # Explicitly set to float16 for T4 GPU
)

print("✅ Base model loaded (using fp16 for T4 compatibility)")

In [ ]:
# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# Setup LoRA
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "k_proj", "v_proj", "dense"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# T4 GPU: Ensure ALL parameters are fp16 (not bf16)
# Convert any bf16 parameters and buffers to fp16
import torch

for name, param in model.named_parameters():
    if param.dtype == torch.bfloat16:
        param.data = param.data.to(torch.float16)
        print(f"Converted parameter {name} from bf16 to fp16")

for name, buffer in model.named_buffers():
    if buffer.dtype == torch.bfloat16:
        buffer.data = buffer.data.to(torch.float16)
        print(f"Converted buffer {name} from bf16 to fp16")

# Verify no bf16 tensors remain
bf16_params = [name for name, p in model.named_parameters() if p.dtype == torch.bfloat16]
bf16_buffers = [name for name, b in model.named_buffers() if b.dtype == torch.bfloat16]

if bf16_params or bf16_buffers:
    print(f"WARNING: Found bf16 tensors: {bf16_params + bf16_buffers}")
else:
    print("✅ All model tensors are fp16 (T4 compatible)")

print("✅ Model prepared with QLoRA (fp16 for T4 GPU)")

## 7. Setup GRPO Trainer

In [ ]:
from trl import GRPOTrainer, GRPOConfig

# Define reward function for GRPO
# This function evaluates the quality of generated completions
# For now, using a simple length-based reward - you can replace with a reward model
def reward_function(prompts, completions, **kwargs):
    """
    Simple reward function for GRPO training.
    Returns rewards for each completion.
    You can replace this with a more sophisticated reward model.
    """
    rewards = []
    for completion in completions:
        # Simple reward: encourage reasonable length (not too short, not too long)
        # You can customize this based on your needs
        length = len(completion.split())
        if length < 5:
            reward = -1.0  # Penalize very short responses
        elif length > 500:
            reward = -0.5  # Slightly penalize very long responses
        else:
            reward = 1.0  # Reward reasonable length responses
        
        rewards.append(reward)
    
    return rewards

# Setup GRPO config
# Note: GRPOConfig extends TrainingArguments, so it uses the same parameters
# IMPORTANT: T4 GPU does NOT support bf16 - must use fp16
# IMPORTANT: per_device_eval_batch_size must be divisible by num_generations
# IMPORTANT: For T4, we use pure fp16 (not mixed precision) to avoid gradient scaler issues
grpo_config = GRPOConfig(
    output_dir=OUTPUT_DIR,
    learning_rate=LEARNING_RATE,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,  # Must be divisible by num_generations
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    lr_scheduler_type="cosine",
    warmup_steps=100,  # Increased back for better training
    logging_steps=LOGGING_STEPS,
    save_steps=SAVE_STEPS,
    eval_steps=SAVE_STEPS,
    eval_strategy="steps",  # Re-enabled evaluation
    save_strategy="steps",
    save_total_limit=3,  # Save more checkpoints
    load_best_model_at_end=True,  # Re-enabled to load best model
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=False,  # T4 GPU: Disable AMP fp16 (we use pure fp16 instead)
    bf16=False,  # T4 GPU: MUST be False (T4 doesn't support bf16)
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    report_to="tensorboard",
    remove_unused_columns=False,
    use_cpu=False,  # Ensure we're using GPU
)

# Note: GRPO generation parameters are controlled internally
# The trainer will use default generation settings

# Initialize GRPO trainer
# Note: GRPOTrainer requires reward_funcs parameter
# Pass tokenizer via processing_class for proper text generation
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,  # Provide tokenizer for generation
    args=grpo_config,
    train_dataset=train_dataset["train"],
    eval_dataset=train_dataset["test"],  # Re-enabled evaluation dataset
    reward_funcs=reward_function,  # Required: reward function(s) for GRPO
)

# T4 GPU: Verify no gradient scaler (pure fp16 mode, not AMP)
print(f"Trainer using AMP fp16: {trainer.args.fp16}")
print(f"Trainer using bf16: {trainer.args.bf16}")
if hasattr(trainer, 'scaler') and trainer.scaler is not None:
    print("⚠️  Gradient scaler detected - this may cause issues")
else:
    print("✅ No gradient scaler (pure fp16 mode for T4)")

# Final T4 check: Ensure model tensors are still fp16 (not bf16)
bf16_found = False
for name, param in model.named_parameters():
    if param.dtype == torch.bfloat16:
        param.data = param.data.to(torch.float16)
        bf16_found = True
for name, buffer in model.named_buffers():
    if buffer.dtype == torch.bfloat16:
        buffer.data = buffer.data.to(torch.float16)
        bf16_found = True

if bf16_found:
    print("⚠️  Converted remaining bf16 tensors to fp16")
else:
    print("✅ All tensors confirmed fp16 (T4 ready)")

print("\n✅ GRPO trainer initialized (T4 compatible - pure fp16 mode)")

## 8. Clear GPU Cache and Prepare for Training

Free up GPU memory before training begins.

In [ ]:
# Clear GPU cache before training to free up memory
import torch
import gc

torch.cuda.empty_cache()
gc.collect()

# Check available memory
if torch.cuda.is_available():
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    allocated_memory = torch.cuda.memory_allocated(0) / 1e9
    cached_memory = torch.cuda.memory_reserved(0) / 1e9
    free_memory = total_memory - allocated_memory
    
    print(f"Total GPU Memory: {total_memory:.2f} GB")
    print(f"Allocated Memory: {allocated_memory:.2f} GB")
    print(f"Cached Memory: {cached_memory:.2f} GB")
    print(f"Free Memory: {free_memory:.2f} GB")
    print("\n✅ GPU cache cleared")

## 9. Train the Model

**Training Configuration (targeting ~12-13GB GPU usage):**
- Batch size: 2 (GRPO generates multiple completions per prompt internally)
- Max sequence length: 512 tokens
- Dataset: 5000 samples (train/eval split)
- LoRA rank: 16
- Gradient accumulation: 16 steps (effective batch size = 32)
- Evaluation: Enabled every 500 steps
- Expected GPU usage: 12-13GB out of 16GB available

**If you get OOM errors**, reduce in this order:
1. `BATCH_SIZE = 1` (most effective)
2. `MAX_SEQ_LENGTH = 256` (if still OOM)
3. `MAX_SAMPLES = 2000` (if still OOM)

This will take several hours. Monitor GPU memory usage in the output above.

In [ ]:
# Start training
print("Starting training...")

train_result = trainer.train()

print("✅ Training complete!")

## 10. Save Model

In [ ]:
# Save final model
print("Saving model...")
trainer.save_model()
tokenizer.save_pretrained(OUTPUT_DIR)

# Save training metrics
metrics = train_result.metrics
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)

print(f"✅ Model saved to {OUTPUT_DIR}")

## 11. Test the Model (Inference)

In [ ]:
# Test the model
model.eval()

test_prompt = "What is machine learning?"
formatted_prompt = f"Human: {test_prompt}\n\nAssistant: "

inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Prompt: {test_prompt}")
print(f"Response: {response.split('Assistant:')[-1].strip()}")

## 12. Push to Hugging Face Hub (Optional)

Uncomment and set your model ID to push the model to Hugging Face Hub.

In [ ]:
# Uncomment to push to Hub
# HUB_MODEL_ID = "your-username/phi2-grpo-finetuned"
# trainer.push_to_hub(hub_model_id=HUB_MODEL_ID)
# print(f"✅ Model pushed to https://huggingface.co/{HUB_MODEL_ID}")